In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k
from src.models import GMFBCEModel, MLPBCEModel, NeuralMatrixFactorizationBCEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = 'ml-100k'
    epochs = 40
    batch_size = 2048
    gmf_embed_size = 16
    mlp_embed_size = 32
    layers = [32, 16, 8]
    lr = 0.001
    b1 = 0.3
    b2 = 0.6
    weight_decay = 0.0001

dataset = Dataset100k(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
model = NeuralMatrixFactorizationBCEModel(dataset.user_count, dataset.item_count, gmf_embed_size=config.gmf_embed_size, mlp_embed_size=config.mlp_embed_size, layers=config.layers, alpha=0.5)

gmf_model = GMFBCEModel(dataset.user_count, dataset.item_count, embed_size=config.gmf_embed_size)
gmf_model.load_state_dict(torch.load("saved_models/gmfbce.pt"))
mlp_model = MLPBCEModel(dataset.user_count, dataset.item_count, embed_size=config.mlp_embed_size, layers=config.layers)
mlp_model.load_state_dict(torch.load("saved_models/mlpbce.pt"))

model.load_pretrained_weights(gmf_model, mlp_model)
del(gmf_model)
del(mlp_model)

# optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(config.b1, config.b2), weight_decay=config.weight_decay)
# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

trainer = Trainer(dataset, model, optimizer, metrics, 
                  epochs=config.epochs, batch_size=config.batch_size,
                  device=device)

In [4]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

Epoch 0: Avg Loss/Batch 0.308001            


HR@1: 0.2873806998939555
HR@5: 0.6256627783669141
HR@10: 0.7720042417815483
NDCG@1: 0.2873806998939555
NDCG@5: 0.4653906911234425
NDCG@10: 0.5133117450220354


Epoch 1: Avg Loss/Batch 0.307314            


HR@1: 0.28207847295864263
HR@5: 0.6224814422057264
HR@10: 0.7762460233297985
NDCG@1: 0.28207847295864263
NDCG@5: 0.4612976597137803
NDCG@10: 0.5113373628812248


Epoch 2: Avg Loss/Batch 0.308059            


...

HR@1: 0.23223753976670203
HR@5: 0.5959703075291622
HR@10: 0.7592788971367974
NDCG@1: 0.23223753976670203
NDCG@5: 0.4213991752525283
NDCG@10: 0.474507502981838


Epoch 38: Avg Loss/Batch 0.282515            


HR@1: 0.23117709437963946
HR@5: 0.5896076352067868
HR@10: 0.7539766702014846
NDCG@1: 0.23117709437963946
NDCG@5: 0.4183579977409337
NDCG@10: 0.4719181918837519


Epoch 39: Avg Loss/Batch 0.281985            


HR@1: 0.22799575821845175
HR@5: 0.5938494167550371
HR@10: 0.76033934252386
NDCG@1: 0.22799575821845175
NDCG@5: 0.4176683503534991
NDCG@10: 0.47183542926737776


In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

22: {'HR@1': 0.2841993637327678, 'HR@5': 0.6224814422057264, 'HR@10': 0.7751855779427359, 'NDCG@1': 0.2841993637327678, 'NDCG@5': 0.4620285833055621, 'NDCG@10': 0.51180795041922}


In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

3: {'HR@1': 0.28950159066808057, 'HR@5': 0.6277836691410392, 'HR@10': 0.7698833510074231, 'NDCG@1': 0.28950159066808057, 'NDCG@5': 0.46715846445285186, 'NDCG@10': 0.5137269743885267}


In [ ]:
torch.save(trainer.model.state_dict(), "saved_models/nmfbce.pt")
# trainer.model.load_state_dict(torch.load("saved_models/nmfbce.pt"))